In [1]:
import geopandas as gpd
import pandas as pd
import rasterio
from rasterio import mask

import shapely
from shapely.geometry import Polygon, Point
from shapely.ops import cascaded_union
import shapely.speedups

shapely.speedups.enable()

/home/luckyw0w/anaconda3/lib/python3.9/site-packages/geopandas/_compat.py:111: UserWarning: The Shapely GEOS version (3.9.1-CAPI-1.14.2) is incompatible with the GEOS version PyGEOS was compiled with (3.10.3-CAPI-1.16.1). Conversions between both will be slow.
  warnings.warn(


In [7]:
df_district_boundaries = gpd.read_file('data/TSDM/District_Boundary.shp')

In [9]:
df_district_boundaries['Dist_Name'].nunique()

33

In [12]:
india_worldpop_raster_2020 = rasterio.open('data/ind_ppp_2020_1km_Aggregated_UNadj.tif')
print('No. of bands:',(india_worldpop_raster_2020.count))
# Reading the first band, filtering negative raster values and visualise data with matplotlib
india_worldpop_raster_2020_tot = india_worldpop_raster_2020.read(1)
india_worldpop_raster_2020_tot[india_worldpop_raster_2020_tot<0] = None

india_worldpop_raster_2020_nonzero = india_worldpop_raster_2020_tot[india_worldpop_raster_2020_tot>0]
population_worldpop = india_worldpop_raster_2020_nonzero[india_worldpop_raster_2020_nonzero > 0].sum()
print('Total population - Telangana (2020): ',round(population_worldpop/1000000000,2),'billion')

No. of bands: 1
Total population - Telangana (2020):  0.04 billion


In [13]:
def get_population_count(vector_polygon,raster_layer):
    gtraster, bound = rasterio.mask.mask(raster_layer, [vector_polygon], crop=True)
    pop_estimate = gtraster[0][gtraster[0]>0].sum()
    return (pop_estimate.round(2))

In [14]:
%%time
df_district_boundaries['population_count_wp'] = df_district_boundaries['geometry'].apply(get_population_count,raster_layer=india_worldpop_raster_2020)


CPU times: user 456 ms, sys: 12.4 ms, total: 469 ms
Wall time: 483 ms


In [15]:
district_population = df_district_boundaries.groupby(['Dist_Name'])['population_count_wp'].sum().round().reset_index().sort_values(by='population_count_wp')


In [17]:
relative_wealth_data = pd.read_csv('data/ind_pak_relative_wealth_index.csv')

In [18]:
def convert_Point(facebook_relative_wealth):
    return Point(facebook_relative_wealth['longitude'],facebook_relative_wealth['latitude'])

relative_wealth_data['geometry'] = relative_wealth_data[['latitude','longitude']].apply(convert_Point,axis=1)
relative_wealth_data = gpd.GeoDataFrame(relative_wealth_data)

/home/luckyw0w/anaconda3/lib/python3.9/site-packages/pandas/core/dtypes/cast.py:122: ShapelyDeprecationWarning: The array interface is deprecated and will no longer work in Shapely 2.0. Convert the '.coords' to a numpy array instead.
  arr = construct_1d_object_array_from_listlike(values)


In [19]:
relative_wealth_data.head(2)

,latitude,longitude,rwi,error,geometry
0,33.897776,70.037842,-0.074,0.624,POINT (70.03784 33.89778)
1,31.118794,66.807861,-0.569,0.368,POINT (66.80786 31.11879)


In [20]:
def get_rwi_mean(vector_polygon,vector_layer):
    pip_mask = vector_layer.within(vector_polygon)
    pip_data = vector_layer.loc[pip_mask]
    mean_val = round(pip_data['rwi'].mean(),2)
    return(mean_val)

In [21]:
def get_rwi_median(vector_polygon,vector_layer):
    pip_mask = vector_layer.within(vector_polygon)
    pip_data = vector_layer.loc[pip_mask]
    mean_val = round(pip_data['rwi'].median(),2)
    return(mean_val)

In [22]:
df_district_boundaries['rwi_mean'] = df_district_boundaries['geometry'].apply(get_rwi_mean,
                                                                              vector_layer=relative_wealth_data)


In [24]:
district_average_rwi = df_district_boundaries.groupby(['Dist_Name'])['rwi_mean'].mean().reset_index().sort_values(by='rwi_mean')


In [25]:
district_population.head(2)

,Dist_Name,population_count_wp
16,Mulugu,302228.0
5,Jayashankar Bhupalpally,424225.0


In [27]:
df_combined = pd.merge(district_average_rwi,district_population,on=['Dist_Name'])

In [28]:
df_combined['weighted'] = df_combined['population_count_wp']*df_combined['rwi_mean']

In [29]:
df_combined.sort_values(by='weighted').to_excel('result/rwi_average.xlsx')

In [30]:
df_district_boundaries['rwi_median'] = df_district_boundaries['geometry'].apply(get_rwi_median,
                                                                              vector_layer=relative_wealth_data)



In [31]:
district_median_rwi = df_district_boundaries.groupby(['Dist_Name'])['rwi_median'].mean().reset_index().sort_values(by='rwi_median')



In [32]:
df_combined = pd.merge(district_median_rwi,district_population,on=['Dist_Name'])

In [33]:
df_combined['weighted'] = df_combined['population_count_wp']*df_combined['rwi_median']

In [34]:
df_combined.sort_values(by='weighted').to_excel('result/rwi_median.xlsx')

In [35]:
adm_name = gpd.read_file('data/TSDM/Mandal_Boundary.shp')  #Mandal (an administrative boundary) Shapefile

relative_wealth_data = pd.read_csv('data/ind_pak_relative_wealth_index.csv')
relative_wealth_data = gpd.GeoDataFrame(relative_wealth_data,                        #Converting fire points into dataframe
                         geometry=gpd.points_from_xy(
                             relative_wealth_data.longitude,
                             relative_wealth_data.latitude),
                         crs=4326)

rwi_telangna = relative_wealth_data.clip(adm_name)                   #Clipping fire points with Telangana boundaries

In [38]:
import rioxarray as rio

df2 = rwi_telangna[['latitude','longitude']]                     #create a dataframe with lat lon values of rwi dataframe
df2['rwi'] = list(rwi_telangna['rwi'])                           #add column of rwi values
da = df2.set_index(['latitude', 'longitude']).to_xarray()        #convert new dataframe to xarray
da = da.set_coords(['latitude', 'longitude'])                    #set lat and lon as coordinates

da.fillna(-9999.0)                                               #fill None values
da = da.rename({'longitude': 'x','latitude': 'y'})               # change lon lat to x  y for exporting xarray to tif


import rasterio
da.rio.to_raster('result/rwioutput.tif')                                #dump xarray to GeoTIFF
crs = rasterio.crs.CRS({"init": "epsg:4326"})    # or whatever CRS you know the image is in    
with rasterio.open('result/rwioutput.tif', mode='r+') as src:     
    #src.transform = transform
    src.crs = crs
    src.close()

/tmp/ipykernel_49377/1679016454.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df2['rwi'] = list(rwi_telangna['rwi'])                           #add column of rwi values
